In [1]:
%%configure -f 
{
"conf":{
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    
        "spark.executor.heartbeatInterval":"10800s",
        "spark.network.timeout":"24h",
    
        "spark.driver.memory": "8G",
        "spark.executor.memory": "8G",
        "spark.executor.cores":"3",
    
        "spark.dynamicAllocation.enabled":"false",
        "spark.ext.h2o.fail.on.unsupported.spark.param":"false",    
        
        "spark.app.name":"msds694"
      }
}

In [2]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1615354225428_0012,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=msds694>

In [3]:
sc.install_pypi_package("h2o-pysparkling-2.4")
sc.install_pypi_package("h2o")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached h2o_pysparkling_2.4-3.32.0.4.post1-py2.py3-none-any.whl
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached future-0.18.2-py3-none-any.whl
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)

  Using cached h2o-3.32.0.4-py2.py3-none-any.whl

In [4]:
final_df=spark.read.parquet("s3a://facebookmsds694/final_df.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from h2o.automl import H2OAutoML
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from pysparkling import *
import h2o

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
final_df.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------------+-----+
|          Text_array|               Title|                Body|            Tags|      Text_array_new|        Text_Vectors|     Tags_joined|count|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+----------------+-----+
|use,condit,given,...|how to use a whil...|<p>i am fetching ...|   php mysql sql|[use, condit, giv...|(20000,[0,1,9,22,...|   php mysql sql| 1649|
|lose,day,random,a...|I lose each day a...|<p>Ok my problem ...|     probability|[lose, day, rando...|(20000,[19,20,30,...|     probability| 3847|
|regard,painleve',...|Regarding Painlev...|<p>Painleve's fir...|complex-analysis|[regard, painleve...|(20000,[2,66,73,9...|complex-analysis| 2911|
|contentel,part,vi...|Why ContentElemen...|<p>I was wonderin...|             wpf|[contentel, part,...|(20000,[0,1,6,41

In [7]:
filtered_df = final_df.withColumn("num_labels",size(split(col('Tags')," ").cast("array<string>"))).filter(col("num_labels")==1)
filtered_df_cache=filtered_df.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
filtered_df_cache.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+
|          Text_array|               Title|                Body|                Tags|      Text_array_new|        Text_Vectors|         Tags_joined|count|num_labels|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+----------+
|lose,day,random,a...|I lose each day a...|<p>Ok my problem ...|         probability|[lose, day, rando...|(20000,[19,20,30,...|         probability| 3847|         1|
|regard,painleve',...|Regarding Painlev...|<p>Painleve's fir...|    complex-analysis|[regard, painleve...|(20000,[2,66,73,9...|    complex-analysis| 2911|         1|
|contentel,part,vi...|Why ContentElemen...|<p>I was wonderin...|                 wpf|[contentel, part,...|(20000,[0,1,6,41,...|                 wpf| 2674|         1|
|cus

In [9]:
filtered_df_cache_text=filtered_df_cache.withColumn("Text",concat(col("Title"),lit(" "),col("Body")))\
                                .select("Text","Tags")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
filtered_df_cache_text.cache()
filtered_df_cache_text.show()
filtered_df_cache.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|                Text|                Tags|
+--------------------+--------------------+
|I lose each day a...|         probability|
|Regarding Painlev...|    complex-analysis|
|Why ContentElemen...|                 wpf|
|custom cell and c...|              vb.net|
|In C#, when shoul...|                  c#|
|How can I define ...|          autohotkey|
|Java - Basic prog...|                java|
|Javascript DropDo...|          javascript|
|Use JavaScript to...|          javascript|
|Programming a var...|              iphone|
|z-indexs for posi...|                 css|
|Adding new roots ...|                 git|
|Could not find co...|             android|
|Why are two empty...|                  c#|
|iPhone , core dat...|              iphone|
|Identify the thre...|           algorithm|
|array functions: ...|               excel|
|onclick="javascri...|          javascript|
|can't have a file...|                java|
|Help identifying ...|programmin

In [11]:
from pysparkling import *
hc = H2OContext.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Connecting to H2O server at http://ip-172-31-5-203.us-west-2.compute.internal:54322 ... successful.
--------------------------  -------------------------------------------------------------------------------------------------------
H2O_cluster_uptime:         11 secs
H2O_cluster_timezone:       UTC
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.32.0.4
H2O_cluster_version_age:    1 month and 8 days
H2O_cluster_name:           sparkling-water-livy_application_1615354225428_0012
H2O_cluster_total_nodes:    2
H2O_cluster_free_memory:    15.81 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  6
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://ip-172-31-5-203.us-west-2.compute.internal:54322
H2O_connection_proxy:       null
H2O_internal_security:      False
H2O_API_Extensions:         XGBoost, Algos, Amazon S3, Sparkling Water REST API Extensions, AutoML, Core V3, TargetEncoder, Core V4
Python_version:             3.6.8 final
----------

In [12]:
filt_h2o= hc.asH2OFrame(filtered_df_cache_text, "filt_h2o")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
[trainingDF, testingDF] = filtered_df_cache_text.randomSplit([0.8, 0.2])
trainingDF.cache()
testingDF.cache()
filtered_df_cache.unpersist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Text_array: string, Title: string, Body: string, Tags: string, Text_array_new: array<string>, Text_Vectors: vector, Tags_joined: string, count: bigint, num_labels: int]

In [14]:
from pysparkling.ml import H2OGBM, H2OWord2Vec
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

tokenizer = RegexTokenizer(inputCol="Text", minTokenLength=2)
stopWordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol())
w2v = H2OWord2Vec(sentSampleRate=0, epochs=10, inputCol=stopWordsRemover.getOutputCol())
gbm = H2OGBM(labelCol="Tags", featuresCols=[w2v.getOutputCol()])

pipeline = Pipeline(stages=[tokenizer, stopWordsRemover, w2v, gbm])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
model = pipeline.fit(trainingDF)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
predictions= model.transform(testingDF)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
predictions_2=hc.asSparkFrame(predictions)
predictions_2.write.parquet("s3a://facebookmsds694/prediction.parquet")

In [ ]:
testingDF.write.parquet("s3a://facebookmsds694/testing.parquet")

In [23]:
type(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [24]:
predictions.write.parquet("s3a://facebookmsds694/prediction.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+-----------------------------------+-------------------------------------+--------------------+--------------------+------------------+
|                Text|               Tags|RegexTokenizer_4c8826e430e4__output|StopWordsRemover_0332b7e641f4__output| detailed_prediction|  H2OWord2Vec_output|        prediction|
+--------------------+-------------------+-----------------------------------+-------------------------------------+--------------------+--------------------+------------------+
| NetworkOnMainMai...|            android|               [networkonmainmai...|                 [networkonmainmai...|[android, [2.4106...|[0.021983212, 0.0...|           android|
| Ten employees, $...|      combinatorics|               [ten, employees,,...|                 [ten, employees,,...|[combinatorics, [...|[0.22891362, -0.1...|     combinatorics|
|"Declaration of x...|                c++|               ["declaration, of...|                 ["declaration, 

In [41]:
predictions.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Text: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- RegexTokenizer_4c8826e430e4__output: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- StopWordsRemover_0332b7e641f4__output: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- detailed_prediction: struct (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- probabilities: struct (nullable = false)
 |    |    |-- .htaccess: double (nullable = false)
 |    |    |-- .net: double (nullable = false)
 |    |    |-- 2007: double (nullable = false)
 |    |    |-- 2010: double (nullable = false)
 |    |    |-- abstract-algebra: double (nullable = false)
 |    |    |-- actionscript-3: double (nullable = false)
 |    |    |-- active-directory: double (nullable = false)
 |    |    |-- ajax: double (nullable = false)
 |    |    |-- algebra-precalculus: double (nullable = false)
 |    |    |-- algebraic-geometry: double (nullable = false)
 |    |    

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
predictionandlabel=predictions.select("prediction",col("Tags").alias("Label"))\
                                .withColumn("Flag",when(col("prediction") == col("Label"), 1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
predictionandlabel.select(sum("Flag")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|sum(Flag)|
+---------+
|    57323|
+---------+

In [48]:
predictionandlabel.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

123647

In [50]:
accuracy= 57323/123647

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
print(accuracy*100,'%')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

46.36020283549136 %

In [53]:
### AUTOML

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
from pysparkling.ml import H2OAutoML
from pysparkling.ml import H2OGBM, H2OWord2Vec
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

tokenizer = RegexTokenizer(inputCol="Text", minTokenLength=2)
stopWordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol())
w2v = H2OWord2Vec(sentSampleRate=0, epochs=10, inputCol=stopWordsRemover.getOutputCol())
gbm = H2OAutoML(labelCol="Tags",maxModels=2,nfolds=2)

pipeline = Pipeline(stages=[tokenizer, stopWordsRemover, w2v, gbm])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
model = pipeline.fit(trainingDF)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o2644.fit.
: java.lang.RuntimeException: H2OContext needs to be created in order to train the model. Please create one as H2OContext.getOrCreate().
	at ai.h2o.sparkling.H2OContext$$anonfun$ensure$1.apply(H2OContext.scala:416)
	at ai.h2o.sparkling.H2OContext$$anonfun$ensure$1.apply(H2OContext.scala:416)
	at scala.Option.getOrElse(Option.scala:121)
	at ai.h2o.sparkling.H2OContext$.ensure(H2OContext.scala:415)
	at ai.h2o.sparkling.ml.features.H2OWord2Vec.fit(H2OWord2Vec.scala:66)
	at ai.h2o.sparkling.ml.features.H2OWord2Vec.fit(H2OWord2Vec.scala:36)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.

In [66]:
model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PipelineModel_b1656755dec3

In [67]:
model.leaderboard

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'PipelineModel' object has no attribute 'leaderboard'
Traceback (most recent call last):
AttributeError: 'PipelineModel' object has no attribute 'leaderboard'

